Import modules

In [15]:
import torch
from torchvision import datasets, transforms
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from torch import linalg as LA


Function to compute your base score 

In [16]:
def compute_base_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score

In [17]:
def compute_runtime_factor(runtime, min_thres, max_thres):
    if runtime <= min_thres:
        score_factor = 1
    elif runtime >= max_thres:
        score_factor = 0
    else:
        score_factor = 0.5

    return score_factor

Function to run your k-NN algorithm and compute its accuracy and runtime

In [18]:
def run(x_train, y_train, x_test, y_test, n_classes, device, n_runs):
    if device != 'cpu' and torch.cuda.is_available():
        device = torch.device("cuda")
        print('Running on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Running on CPU')

    run_times = []

    for i in range(n_runs):
        start = timeit.default_timer()
        # np.random.seed(0)
        predicted_y_test = knn(x_train, y_train, x_test, n_classes, device)
        # np.random.seed()
        stop = timeit.default_timer()
        run_time = stop - start
        run_times.append(run_time)

        print(f'run {i + 1} : run_time: {run_time}')

    assert isinstance(predicted_y_test, np.ndarray), "predicted test labels must be returned as a numpy array"

    correct_predict = (y_test
                       == predicted_y_test).astype(np.int32).sum()
    accuracy = float(correct_predict) / len(y_test)

    run_time = min(run_times)

    print('Correct Predictions: {}/{} total \tAccuracy: {:5f} \tTime: {:2f}'.format(correct_predict,
                                                                                    len(y_test), accuracy, run_time))
    return correct_predict, accuracy, run_time

TODO: Implement knn here

In [19]:
def knn(x_train, y_train, x_test, n_classes, device):
    """
    x_train: 60000 x 784 matrix: each row is a flattened image of an MNIST digit
    y_train: 60000 vector: label for x_train
    x_test: 1000 x 784 testing images
    n_classes: no. of classes in the classification task
    device: pytorch device on which to run the code
    return: predicted y_test which is a 1000-sized vector
    """

    # get the shape size
    num_row_train, num_col_train = x_train.shape
    num_row_test, num_col_test = x_test.shape

    # initialize the output y_test
    y_test = np.zeros(num_row_test)

    # convert to float type
    x_test = x_test.astype(np.float32)
    x_train = x_train.astype(np.float32)

    # Convert from numpy to pytorch tensor
    # Move the data to gpu
    x_test_tt = torch.from_numpy(x_test).to(device)
    x_train_tt = torch.from_numpy(x_train).to(device)
    distance = torch.zeros(num_row_train).to(device)

    # For every test data point, do classification
    for i in list(range(0, num_row_test)):
      
      distance_vector = x_train_tt - x_test_tt[i].repeat(num_row_train, 1)
      distance = LA.norm(distance_vector, dim=1)
      
      dist, distIndex = torch.topk(distance, 3, largest=False)
      
      closestY = torch.zeros(3)

      # get the labels of closest data points
      for k in list(range(0, 3)):
        closestY[k] = y_train[distIndex[k]]

      # use the mode label
      mode, modeIndex = torch.mode(closestY)
      y_test[i] = mode

    return y_test


Main function

In [20]:
def main():
    min_acc_thres = 0.84
    max_acc_thres = 0.94

    min_runtime_thres = 12
    max_runtime_thres = 24

    n_runs = 5

    n_classes = 10
    # change to cpu to run on CPU
    device = 'gpu'

    mnist_train = datasets.MNIST('data', train=True, download=True,
                                 transform=transforms.Compose([
                                     transforms.Normalize((0.1307,), (0.3081,)),
                                 ])
                                 )
    mnist_test = datasets.MNIST('data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.Normalize((0.1307,), (0.3081,)),
                                ])
                                )
    # convert pytorch tensors to numpy arrays
    (x_train, y_train) = (mnist_train.data.cpu().numpy(), mnist_train.targets.cpu().numpy())
    (x_valid, y_valid) = (mnist_test.data.cpu().numpy(), mnist_test.targets.cpu().numpy())

    # flatten 28x28 images into 784 sized vectors
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_valid = x_valid.reshape(x_valid.shape[0], -1)

    # You may want to use a smaller training set to save time when debugging
    # i.e.: Put something like:
    # (x_train, y_train) = (x_train[:5000], y_train[:5000])

    # For this assignment, we only test on the first 1000 samples of the test set
    (x_valid, y_valid) = (x_valid[:1000], y_valid[:1000])

    print("Dimension of dataset: ")
    print("Train:", x_train.shape, y_train.shape, "\nTest:", x_valid.shape, y_valid.shape)

    (correct_predict, accuracy, run_time) = run(x_train, y_train, x_valid, y_valid, n_classes, device, n_runs)
    base_score = compute_base_score(accuracy, min_acc_thres, max_acc_thres)

    runtime_factor = compute_runtime_factor(run_time, min_runtime_thres, max_runtime_thres)

    overall_score = base_score * runtime_factor

    result = OrderedDict(correct_predict=correct_predict,
                         accuracy=accuracy,
                         run_time=run_time,
                         base_score=base_score,
                         overall_score=overall_score
                         )

    with open('result.txt', 'w') as f:
        f.writelines(pformat(result, indent=4))

    print(pformat(result, indent=4))

Call the main function. You can only run this after filling the _knn_ function above

In [21]:
main()

Dimension of dataset: 
Train: (60000, 784) (60000,) 
Test: (1000, 784) (1000,)
Running on GPU: Tesla T4
run 1 : run_time: 4.226465571999938
run 2 : run_time: 4.20017830200004
run 3 : run_time: 4.204567190000034
run 4 : run_time: 4.197855642000036
run 5 : run_time: 4.208544285000016
Correct Predictions: 962/1000 total 	Accuracy: 0.962000 	Time: 4.197856
OrderedDict([   ('correct_predict', 962),
                ('accuracy', 0.962),
                ('run_time', 4.197855642000036),
                ('base_score', 100.0),
                ('overall_score', 100.0)])
